## Importar librerias y cargar datos

In [10]:
import pandas as pd 
import sqlalchemy
import missingno as msn 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import numpy as np

df = pd.read_csv('../../Data/df.csv')
#DROP COLUMNS
df = df.drop(columns=['adjustSquareMeterPrice', 'automaticACMPrice', 'created_at', 'terraceValue',
                       'utilityRoomValue', 'preofferAcmHigherValue','preofferAcmLowerValue','buildingFloors','estateFloors',
                       'rentValue','garageType','terraceArea','zone','garageValue', 'type'])
def preprocesamiento(df):
    #Drop rows with null values in Y variable
    df = df[df.finalACMPrice.notna()]
    #Calculate Antiguedad of a property in years
    df['Antiguedad'] = date.today().year - df['year']
    df['Antiguedad'] = df['Antiguedad'].astype(int)
    df = df.drop(columns=['id', 'purchaseCaseId', 'year'])
    #delete rows with null values in stratum variables
    df = df[df.stratum.notna()]
    #Turn floor to integer
    df = df[df.floor.notna()]
    df['floor'] = df['floor'].astype(int)
    #df = df[df.stratum.notna()]
    # Convert hasElevator to categorical
    df['hasElevator'] = df['hasElevator'].astype('category')
    return df
df = preprocesamiento(df)

C:\Users\User\AppData\Local\Temp\ipykernel_11608\1282984739.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Antiguedad'] = date.today().year - df['year']
C:\Users\User\AppData\Local\Temp\ipykernel_11608\1282984739.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Antiguedad'] = df['Antiguedad'].astype(int)


## Definir variables X y Y

In [11]:
X = df.drop(columns=['finalACMPrice'])
y = df['finalACMPrice']

In [12]:
#Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
#Train LightGBMRegresor model
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

#Fit model
lgbm = LGBMRegressor(n_estimators=258, learning_rate=0.01, random_state=42)
lgbm.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.01, n_estimators=258, random_state=42)

In [14]:
#Predict
y_pred = lgbm.predict(X_test)
#Evaluate
mae_kmeans = mean_absolute_error(y_test, y_pred)

#Print results in currency format
print('MAE Kmeans: ${:,.2f}'.format(mae_kmeans))

MAE Kmeans: $28,240,646.73
